In [6]:
import sagemaker
from sagemaker.processing import ScriptProcessor

xgboost_container_uri = sagemaker.image_uris.retrieve("xgboost", 'eu-west-1', "1.7-1")

evaluation_processor = ScriptProcessor(
    image_uri=xgboost_container_uri,
    command=["python3"],
    instance_type='ml.m5.large',
    instance_count=1,
    base_job_name="evaluate",
    role=sagemaker.get_execution_role(),
)

In [19]:
import import_ipynb
import c2_training_step

from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.workflow.steps import ProcessingStep

#training_step = c2_training_step.create_training_step()

# define inputs
#model_processing_input = ProcessingInput(source=training_step.properties.ModelArtifacts.S3ModelArtifacts, destination="/opt/ml/processing/model")
model_processing_input = ProcessingInput(source='s3://ktzouvan-trading-point-sagemaker-poc/models/pipelines-3ol04e0v16ml-training-step-sALSc8gqXi/output/', destination="/opt/ml/processing/model")
data_processing_input = ProcessingInput(source='s3://ktzouvan-trading-point-sagemaker-poc/datasets/marketing/', destination="/opt/ml/processing/test")

# define output
evaluation_processing_output = ProcessingOutput(output_name="evaluation", source="/opt/ml/processing/evaluation", destination='s3://ktzouvan-trading-point-sagemaker-poc/models/pipelines-3ol04e0v16ml-training-step-sALSc8gqXi/output/')


INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


In [20]:
evaluation_processor.run(
    code='d1_post_processing.py',
    arguments=['--model_path', '/opt/ml/processing/model','--model_file', 'model.tar.gz','--data_path', '/opt/ml/processing/test', '--output_path', '/opt/ml/processing/evaluation'],
    inputs=[model_processing_input,  data_processing_input],
    outputs=[evaluation_processing_output]
)

INFO:sagemaker:Creating processing-job with name evaluate-2024-09-01-10-41-02-480


...............creating temp folder
loading model:/opt/ml/processing/model/model.tar.gz
Saving classification report to /opt/ml/processing/evaluation/0_evaluation.json

